## Begin by importing the proper Library:

    The psycopg2 library enables a user to create a connection to a localhost PostgreSQL instance.

In [ ]:
import psycopg2

The first thing to establish is a connection to the database. This is performed with the .connect method against the library just imported. The arguments we need to pass to the connection method is the name of the user, password, host address, port, and the name of the database, which is "testdb". I store all of this information in the "conn" variable. As a connection object, "conn" has a method called cursor. This cursor is where SQL is written to run against the local database. I store this object in "cur".

The information I am loading into the database is the collegiate cycling directory. The link is provided below for reference. In this example, the contents of that website are saved in a csv.

In [ ]:
conn = psycopg2.connect(user = "postgres",
                          password = "postgrespass",
                          host = "127.0.0.1",
                          port = "5432",
                          database = "testdb")
cur = conn.cursor()

https://www.nationalmtb.org/collegiate-cycling-directory/ :: Accessed on June 2020.

With the connection live, I now need to ensure the table I want to make doesn't already exist. In my case, the table I want to make will be in the database because I have run this code many times. If I try to make an existing table, I will be met with an error message. In order to work around that, I create a try/except to drop any pre-existing table. The .execute syntax is how all the SQL will be written to run against the database.

Now I create a table in my "testdb" database. This table is called "cycleInfo". I run the .commit method against the connection in order to make sure that the changes just made - i.e. the table that has just been created - are saved to the database. The table contains two text columns: "conference" and "college".

In [ ]:
try:
    cur.execute('DROP TABLE cycleInfo')
except:
    pass

cur.execute('CREATE TABLE cycleInfo (conference varchar(10) NOT NULL,\
                                    college varchar(100) NOT NULL);')
conn.commit()

To get the contents of the csv into the table, I perform the COPY FROM command in SQL. The syntax is this: COPY (table name) FROM (location of file) (type of file). I perform this action against my cursor object using the .execute method and save those changes with the .commit.

Following, I once again use .execute in order to select all of the data from the cycleInfo table. To collect all of the output of an execute method, I run the .fetchall method against the cursor immediately after the .execute method has run. What is returned from .fetchall is stored in "record". "record", as shown below, is a List.

In [ ]:
cur.execute("COPY cycleInfo FROM 'C:\\Users\\Public\\cycle.csv' CSV")
conn.commit()

cur.execute("SELECT * FROM cycleInfo")
record = cur.fetchall()
type(record)

To see what is my newly created List, I iterate through the object and output the contents of what has been copied from the database. Each "i" in "record" represents a row in a database. Thus, each "i" has two pieces of information: the conference and the college.

In [ ]:
for i in record:
    print(i[0] + '\t' + i[1])

I finish here by performing some standard SQL against the database. I am able to perform the query in jupyter notebook with the .execute method as if I was using PGAdmin.

In [ ]:
cur.execute('SELECT COUNT(conference), conference \
            FROM cycleInfo \
            WHERE conference LIKE \'ACCC\' \
            GROUP BY conference \
            ORDER BY COUNT(conference) DESC')
c = cur.fetchall()

for i in c:
    print(i)

Before closing out of the jupyter notebook, I run the following .close methods on the cursor and connection objects to ensure the connection is broken. Not doing so opens me up to memory leakage. 

In [ ]:
cur.close()
conn.close()